# Feature Engineering

This notebook shows the pipeline for the feature enginnering we made in order to get our data ready for the models

## 0. Import Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# to read data from a google sheets
from google.colab import auth
import gspread
from google.auth import default

# authenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## 1 Load Data

In [6]:
def load_data(url, sheet_name):

    sh = gc.open_by_url(url)
    worksheet = sh.get_worksheet(0)

    if sheet_name:
        worksheet = sh.worksheet(sheet_name)
    else:
        worksheet = sh.get_worksheet(0)

    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])

    return df

In [7]:
def clean_time_column(df, column_name):

    time_pattern = re.compile(r'^\d{1,2}:\d{2}:\d{2} (?:AM|PM)$')
    df[column_name] = df[column_name].apply(lambda x: '00:00:00 AM' if not time_pattern.match(x) else x)

    return df

In [8]:
def transform_data_types(df):

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time

    #df['time'] = df['time'].dt.round('H').dt.time
    #df['time']

    # Drop columns
    df = df.drop(columns={
        'Temperature', 'Wind Speed', 'Barometer', 'Visibility'
    })

    # Convert 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Convert 'temperature_celsius' to float
    df['temperature_celsius'] = pd.to_numeric(df['temperature_celsius'], errors='coerce')

    # Convert 'wind_speed_km' to float after removing commas
    df['wind_speed_km'] = pd.to_numeric(df['wind_speed_km'], errors='coerce')

    # Convert 'humidity' to integer
    df['humidity'] = pd.to_numeric(df['humidity'], errors='coerce')

    # Convert 'barometer_mbar' to integer
    df['barometer_mbar'] = pd.to_numeric(df['barometer_mbar'].str.replace(',', ''), errors='coerce')

    # Convert 'visibility' to integer
    df['visibility'] = pd.to_numeric(df['visibility'], errors='coerce')

    return df

### Load and Apply Transformations to Data

In [9]:
url = 'https://docs.google.com/spreadsheets/d/1FfyXJTWHup3Iiz_dUOnGhQ0Zix0Cox4pOzb1fIISlrc/edit#gid=919748997'
sheet_name = 'cleaned_tigre_weather'

df = load_data(url, sheet_name)
df = clean_time_column(df, 'time')
df = transform_data_types(df)
df.head()

,date,time,temperature_celsius,weather,wind_speed_km,wind_direction,humidity,barometer_mbar,visibility
0,2022-01-01,16:00:00,23,Broken clouds.,15,Wind blowing from 20° North-northeast to South...,0.83,1004,NaN
1,2022-01-01,23:00:00,22,Clear.,4,Wind blowing from 0° North to South,0.83,1005,16.0
2,2022-01-01,22:00:00,23,Clear.,7,Wind blowing from 330° North-northwest to Sout...,0.78,1004,16.0
3,2022-01-01,21:00:00,23,Passing clouds.,7,Wind blowing from 0° North to South,0.78,1004,NaN
4,2022-01-01,20:00:00,24,Passing clouds.,11,Wind blowing from 10° North to South,0.74,1004,NaN


## 2. Weather Categorization

Weather column has almost 29 types of weather classification. This function optimizes and categorize them in just 6 weather types

In [11]:
def reduce_weather_categories(df):
    # Create a mapping dictionary to group similar categories
    weather_mapping = {
        'Passing clouds.': 'Cloudy',
        'Clear.': 'Clear',
        'Sunny.': 'Clear',
        'Partly sunny.': 'Cloudy',  # Group Partly Cloudy into Cloudy
        'Scattered clouds.': 'Cloudy',  # Group Partly Cloudy into Cloudy
        'Overcast.': 'Cloudy',
        'Partly cloudy.': 'Cloudy',  # Group Partly Cloudy into Cloudy
        'Light rain. Overcast.': 'Light Rain',
        'Broken clouds.': 'Cloudy',  # Group Partly Cloudy into Cloudy
        'Fog.': 'Fog',
        'Light rain. Mostly cloudy.': 'Light Rain',
        'Mostly cloudy.': 'Cloudy',
        'Light rain. Clear.': 'Light Rain',
        'Rain. Fog.': 'Rainy',
        'More clouds than sun.': 'Cloudy',
        'Low clouds.': 'Cloudy',
        'Rain. Scattered clouds.': 'Rainy',
        'Light rain. Partly sunny.': 'Light Rain',
        'Thunderstorms. Passing clouds.': 'Thunderstorms',
        'Drizzle. Overcast.': 'Light Rain',
        'Drizzle. Low clouds.': 'Light Rain',
        'Light rain. Passing clouds.': 'Light Rain',
        'Thunderstorms. Broken clouds.': 'Thunderstorms',
        'Rain. Mostly cloudy.': 'Rainy',
        'Rain. Overcast.': 'Rainy',
        'Light rain. Fog.': 'Light Rain',
        'Cloudy.': 'Cloudy',
        'Heavy rain. Mostly cloudy.': 'Rainy',
        'Thunderstorms. Overcast.': 'Thunderstorms'
    }

    # Map the values using the mapping dictionary
    df['weather'] = df['weather'].map(weather_mapping).fillna(df['weather'])

    # Group Drizzle into Light Rain
    df['weather'] = df['weather'].replace('Drizzle', 'Light Rain')
    df['weather'] = df['weather'].replace('Partly Cloudy','Cloudy')

    return df

# Call the function to reduce categories
df = reduce_weather_categories(df)
df['weather'].value_counts()

Cloudy           14599
Clear             9395
Light Rain        1231
Rainy              402
Fog                216
Thunderstorms      124
Name: weather, dtype: int64

## 3. Wind Categorization

This function categorize wind directions by its degree

In [12]:
def categorize_wind_direction(degree):

    direction_mapping = {
        (0, 22.5): 'North',
        (22.5, 67.5): 'Northeast',
        (67.5, 112.5): 'East',
        (112.5, 157.5): 'Southeast',
        (157.5, 202.5): 'South',
        (202.5, 247.5): 'Southwest',
        (247.5, 292.5): 'West',
        (292.5, 337.5): 'Northwest',
        (337.5, 360): 'North'
    }

    for key, value in direction_mapping.items():
        if key[0] <= degree < key[1]:
            return value

# Extract the degree from the wind direction and apply the categorize_wind_direction function
df['degree'] = df['wind_direction'].str.extract(r'(\d+)').astype(float)
df['Cleaned Wind Direction'] = df['degree'].apply(categorize_wind_direction)

df.head()

,date,time,temperature_celsius,weather,wind_speed_km,wind_direction,humidity,barometer_mbar,visibility,degree,Cleaned Wind Direction
0,2022-01-01,16:00:00,23,Cloudy,15,Wind blowing from 20° North-northeast to South...,0.83,1004,NaN,20.0,North
1,2022-01-01,23:00:00,22,Clear,4,Wind blowing from 0° North to South,0.83,1005,16.0,0.0,North
2,2022-01-01,22:00:00,23,Clear,7,Wind blowing from 330° North-northwest to Sout...,0.78,1004,16.0,330.0,Northwest
3,2022-01-01,21:00:00,23,Cloudy,7,Wind blowing from 0° North to South,0.78,1004,NaN,0.0,North
4,2022-01-01,20:00:00,24,Cloudy,11,Wind blowing from 10° North to South,0.74,1004,NaN,10.0,North


## 4. Merge Alerta Tigre Data

We manually collected data from official Tigre social media accounts to determine where "alertas" were flagged for a particular day.

In [13]:
tigre = load_data(url='https://docs.google.com/spreadsheets/d/1pw5Z4gH9zG-CF2z6tR733f10RBhG3fSj5kZTltlOkEk/edit#gid=229557500', sheet_name='ALERTA_TIGRE')
tigre.head()

,date,time,alerta_crecida
0,13/3/2024,15:05,YES
1,12/3/2024,19:30,YES
2,5/3/2024,8:00,YES
3,4/3/2024,3:30,YES
4,3/3/2024,7:30,YES


In [14]:
tigre.shape

(194, 3)

Tihs function merges alerta data to weather data by date. For the rest of the rows that doesn't match the alerta, we add "NO" categories

In [15]:
def merge_alerta_to_weather(alerta_tigre, weather, new_dataframe=None):

    # Convert the 'date' column in both DataFrames to datetime format
    alerta_tigre['date'] = pd.to_datetime(alerta_tigre['date'], dayfirst=True)
    weather['date'] = pd.to_datetime(weather['date'], dayfirst=True)

    # Merge the DataFrames based on the 'date' column
    merged_df = pd.merge(weather, alerta_tigre[['date', 'alerta_crecida']], on='date', how='left')

    # Fill in the 'alerta_crecida' column with 'NO' for dates not present in alerta
    merged_df['alerta_crecida'].fillna('NO', inplace=True)

    # Return the merged DataFrame or assign it to a new variable if requested
    if new_dataframe:
        merged_df.to_csv(new_dataframe, index=False)
    else:
        return merged_df

# Example usage without saving to a file
merged_df = merge_alerta_to_weather(alerta_tigre=tigre, weather=df)

# Example usage with saving to a file
#merge_alerta_to_weather(alerta_tigre=tigre, weather=df, new_dataframe='merged_data.csv')

merged_df.head()

,date,time,temperature_celsius,weather,wind_speed_km,wind_direction,humidity,barometer_mbar,visibility,degree,Cleaned Wind Direction,alerta_crecida
0,2022-01-01,16:00:00,23,Cloudy,15,Wind blowing from 20° North-northeast to South...,0.83,1004,NaN,20.0,North,NO
1,2022-01-01,23:00:00,22,Clear,4,Wind blowing from 0° North to South,0.83,1005,16.0,0.0,North,NO
2,2022-01-01,22:00:00,23,Clear,7,Wind blowing from 330° North-northwest to Sout...,0.78,1004,16.0,330.0,Northwest,NO
3,2022-01-01,21:00:00,23,Cloudy,7,Wind blowing from 0° North to South,0.78,1004,NaN,0.0,North,NO
4,2022-01-01,20:00:00,24,Cloudy,11,Wind blowing from 10° North to South,0.74,1004,NaN,10.0,North,NO


## 5. Merge Tide Data

We collected this data from the official hidrography service of San Fernando

In [16]:
tides= load_data(url='https://docs.google.com/spreadsheets/d/1EFyq0lda1jpICPOuDtePYMxiKjRngkVODIgGlBnnZn0/edit#gid=119751095', sheet_name='tide_tigre_cleaned')
tides= tides.drop(columns={'HEIGH'})
tides.head()

,DATE,TIME,HEIGH_m
0,2022-01-01,5:55,1.37
1,2022-01-01,13:34,0.52
2,2022-01-01,18:30,1.07
3,2022-01-02,0:46,0.57
4,2022-01-02,6:47,1.31


This function rounds time data so it's easier to merge dataframes later

In [17]:
def convert_types_tides(tides):

    tides.rename(columns={'DATE': 'date'}, inplace=True)

    tides['date'] = pd.to_datetime(tides['date'])

    # Convert 'TIME' column to datetime
    tides['TIME'] = pd.to_datetime(tides['TIME'])

    # Round the times in the 'tides' dataframe to the nearest hour
    tides['time'] = tides['TIME'].dt.round('H').dt.time

    # Convert 'HEIGH_m' column to float
    tides['HEIGH_m'] = tides['HEIGH_m'].astype(float)

    tides = tides[['date', 'time', 'HEIGH_m']]

    return tides

convert_types_tides(tides)

,date,time,HEIGH_m
0,2022-01-01,06:00:00,1.37
1,2022-01-01,14:00:00,0.52
2,2022-01-01,18:00:00,1.07
3,2022-01-02,01:00:00,0.57
4,2022-01-02,07:00:00,1.31
...,...,...,...
4214,2024-12-30,21:00:00,1.14
4215,2024-12-31,03:00:00,0.81
4216,2024-12-31,07:00:00,1.01
4217,2024-12-31,15:00:00,0.46


In [ ]:
tides.time.count()

4219

This function adjust time to merge all possible row times

In [27]:
def adjust_time(df):
  merged_df['time'] = merged_df['time'].astype(str)
  merged_df['time'] = pd.to_datetime(merged_df['time'])
  merged_df['time'] = merged_df['time'].dt.round('H').dt.time

  return df

adjust_time(merged_df)

0        16:00:00
1        23:00:00
2        22:00:00
3        21:00:00
4        20:00:00
           ...   
25962    16:00:00
25963    17:00:00
25964    00:00:00
25965    07:00:00
25966    19:00:00
Name: time, Length: 25967, dtype: object

Finally, we merge data by time and date.  
  
Note that we apply the last tide heigh observation for those rows that didn't have values.

In [28]:
def merge_and_fill(weather_df, tides_df):

    # Merge the data frames on 'date' and 'time' columns with left join
    merged_df = pd.merge(weather_df, tides_df, on=['date', 'time'], how='left')

    # Sort the merged dataframe by 'date' and 'time' columns
    merged_df.sort_values(by=['date', 'time'], inplace=True)

    # Fill NaN values in 'HEIGH_m' column with the last observed tide height
    merged_df['HEIGH_m'] = merged_df['HEIGH_m'].fillna(method='ffill')

    # Convert 'TIME' column to string and extract only the time component
    merged_df['TIME'] = merged_df['TIME'].dt.time

    return merged_df

merged_data = merge_and_fill(merged_df, tides)

In [29]:
merged_data['TIME'].count()

4144

## 6. Final Dataframe for Model

In [ ]:
final_df  = merged_data[['date','time','temperature_celsius','weather','humidity','barometer_mbar','visibility','degree','wind_speed_km','Cleaned Wind Direction','alerta_crecida','TIME','HEIGH_m']]
final_df = final_df.rename(columns={'TIME': 'time_crecida','Cleaned Wind Direction':'wind_direction','HEIGH_m':'heigh_m'})
final_df

,date,time,temperature_celsius,weather,humidity,barometer_mbar,visibility,degree,wind_speed_km,wind_direction,alerta_crecida,time_crecida,heigh_m
16,2022-01-01,00:00:00,28,Clear.,0.55,1004,16.0,100.0,24,East,NO,NaT,NaN
17,2022-01-01,01:00:00,28,Clear.,0.58,1003,16.0,90.0,17,East,NO,NaT,NaN
18,2022-01-01,02:00:00,27,Clear.,0.66,1003,16.0,70.0,13,East,NO,NaT,NaN
19,2022-01-01,03:00:00,27,Passing clouds.,0.70,1002,NaN,40.0,13,Northeast,NO,NaT,NaN
22,2022-01-01,04:00:00,27,Passing clouds.,0.70,1001,NaN,20.0,17,North,NO,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25964,2024-12-31,17:00:00,25,Sunny.,0.69,1007,16.0,100.0,22,East,NO,NaT,0.46
25967,2024-12-31,19:00:00,25,Passing clouds.,0.69,1010,NaN,50.0,9,Northeast,NO,NaT,0.46
25950,2024-12-31,20:00:00,25,Clear.,0.74,1010,16.0,90.0,9,East,NO,NaT,0.46
25949,2024-12-31,21:00:00,23,Clear.,0.83,1010,8.0,130.0,9,Southeast,NO,21:12:00,1.24


## 7. Save csv File

In [ ]:
final_df.to_csv('tigre_dataset.csv')